# goal of notebook
- import ground truth json file created by thesaurus_manual_annotation_dataset_parsing.ipynb
- import thema nodes that contain theme codes, name, descriptions and hierarchy
- try to align with exact match or levenstein
- compute alignement performance

In [1]:
import json
import sys
sys.path.append('../')
sys.path.append('../andre')
import andre.utils as utils

# ground truth import

In [2]:
file_path = '../book_themes_alignment_data/themes_validation_set_ground_truth.json'
# Open the JSON file and load its contents into a dictionary
with open(file_path, "r") as file:
    themes_validation_set_ground_truth = json.load(file)

themes_validation_set_ground_truth

{'balle, ballon': ['SF', 'SF'],
 'animaux domestiques': ['WNG', 'WNG'],
 'sénégal': ['1HFDS', '1HFDS'],
 'livre-jeu': ['YBCS2', 'YBG'],
 "atelier d'écriture": ['CBW', 'CBW'],
 'edition': ['KNTP1', 'KNTP1'],
 'ours': ['WNCF', 'YNNJV23'],
 'théâtre contemporain': ['DDC', 'DDC'],
 'maltraitance': ['YXQD', 'YXQD'],
 'transhumanisme': ['UBJ', 'FLP', 'UBJ'],
 'genre': ['JBSF', 'YXC'],
 'alaska': ['1KBB-US-WPNA', '1KBB-US-WPNA'],
 'biographie': ['DNB', 'YNB'],
 'bolivie': ['1KLSL', '1KLSL'],
 'conte détourné, le loup et les 7 cabris': ['YFHG', 'YFHG'],
 'aventure et mystère': ['YFCF', 'YFC', 'YNX'],
 'conte détourné': ['YFHG', 'YFHG'],
 'yémen': ['1FBXY', '1FBXY'],
 'hippocampe': ['WNCS1', 'N/A'],
 'cow-boy': ['FJW', 'FRJ'],
 'dragon': ['VXQM1', 'YNXB1'],
 'père noël': ['5HPD', 'N/A'],
 'sida': ['MJCJ2', 'MJCJ2'],
 'souvenir/mémoire': ['JMR', 'VSPT'],
 'pompier': ['JKSW2', 'JKSW2'],
 'iran': ['1FBN', '1FBN'],
 'semaine': ['YBLJ', '5HCK'],
 'finlande': ['1DNF', '1DNF'],
 'peintre': ['AGB', 'YN

In [3]:
list(themes_validation_set_ground_truth.keys())

['balle, ballon',
 'animaux domestiques',
 'sénégal',
 'livre-jeu',
 "atelier d'écriture",
 'edition',
 'ours',
 'théâtre contemporain',
 'maltraitance',
 'transhumanisme',
 'genre',
 'alaska',
 'biographie',
 'bolivie',
 'conte détourné, le loup et les 7 cabris',
 'aventure et mystère',
 'conte détourné',
 'yémen',
 'hippocampe',
 'cow-boy',
 'dragon',
 'père noël',
 'sida',
 'souvenir/mémoire',
 'pompier',
 'iran',
 'semaine',
 'finlande',
 'peintre',
 'relation enfant, adulte',
 'papillon',
 'souffrance',
 'nuit',
 'guatemala',
 'préhistoire',
 'astronomie',
 'révolte',
 'naissance/maternité',
 'documentaire',
 'oman',
 'univers/système solaire',
 'écrivain, écriture',
 'histoire : 18e siècle',
 'débarquement',
 'aide humanitaire',
 'rugby',
 'histoire sans texte',
 'poisson',
 "lecture de l'image",
 'roller',
 'licorne',
 'regard des autres',
 'arts martiaux',
 'recherche de ses origines',
 'tchécoslovaquie',
 'eau',
 'londres',
 'hygiène/propreté',
 'monde arabe',
 "conte détourné

# thema nodes import

In [5]:
# Open the JSON file and load its contents into a dictionary
with open("../book_themes_alignment_data/thema_code_dict.json", "r") as file:
    thema_code_dict = json.load(file)

thema_code_dict

{'A': {'CodeValue': 'A',
  'CodeDescription': 'Arts',
  'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'Arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'Théorie de l’art',
  'CodeNotes': 'Lié : QDTN',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'Conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'Utilisez tous le

In [6]:
thema_code_dict['A']['CodeNotes']

'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*'

# token number calculation for LLMs

In [10]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
num_tokens = 0

for theme in thema_code_dict.values():
    theme_code = str(theme["CodeValue"])
    depth = utils.get_thema_node_depth(theme_code)
    if depth <= 3:
        num_tokens += utils.num_tokens_from_string(theme_code, "cl100k_base")
        num_tokens += utils.num_tokens_from_string(theme["CodeDescription"], "cl100k_base")
    # num_tokens += utils.num_tokens_from_string(theme["CodeNotes"], "cl100k_base")

print(num_tokens)


43289


In [3]:
utils.num_tokens_from_string("hello world I love trains!", "cl100k_base")

6

# preprocessing: lower descriptions

In [8]:
lowered_thema_codeDescriptions = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    theme_name = theme["CodeDescription"]
    preprocessed_description = theme_name.lower()    
    lowered_thema_codeDescriptions[code] = preprocessed_description

lowered_thema_codeDescriptions


{'A': 'arts',
 'AB': 'arts : généralités',
 'ABA': 'théorie de l’art',
 'ABC': 'conservation, restauration et entretien d’œuvres d’art',
 'ABK': 'contrefaçon, falsification et vol d’œuvres d’art',
 'ABQ': 'art : aspects financiers',
 'AF': 'arts : formes d’art',
 'AFC': 'peinture et tableaux',
 'AFCC': 'tableaux, aquarelle et pastels',
 'AFCL': 'tableaux et peinture à l’huile',
 'AFCM': 'peintures murales et fresques',
 'AFCP': 'tableaux et peinture à l’encre',
 'AFF': 'dessin et dessins',
 'AFFC': 'dessin et dessins au crayon, fusain ou pastel',
 'AFFK': 'dessin et dessins à la plume, au pinceau et à l’encre',
 'AFH': 'estampes et gravure',
 'AFJ': 'autres formes d’art graphique ou visuel',
 'AFJY': 'body art et tatouage',
 'AFK': 'formes d’art non graphique et électroniques',
 'AFKB': 'sculpture',
 'AFKC': 'sculptures, masques, reliefs',
 'AFKG': 'métal précieux, pierres précieuses et bijoux : techniques et procédés',
 'AFKN': 'installation artistique',
 'AFKP': 'performance artistiq

In [9]:
lowered_thema_dict = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    lowered_thema_dict[code] = theme
    lowered_thema_dict[code]["CodeDescription"] = theme["CodeDescription"].lower()
    lowered_thema_dict[code]["CodeNotes"] = theme["CodeNotes"].lower()

lowered_thema_dict

{'A': {'CodeValue': 'A',
  'CodeDescription': 'arts',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'arts : généralités',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'théorie de l’art',
  'CodeNotes': 'lié : qdtn',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'conservation, restauration et entretien d’œuvres d’art',
  'CodeNotes': 'utilisez tous le

# basic theme matching
match if source theme name contained in thema theme name

In [10]:
def exact_match_evaluation(thema_dict):
    theme_evaluation_exact = {}
    match_count = 0
    matches = []
    all_data = []

    for theme_to_evaluate in themes_validation_set_ground_truth.keys():
        all_data.append(theme_to_evaluate)
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"]
            if theme_to_evaluate == thema_value and theme_to_evaluate not in theme_evaluation_exact.values():
                matches.append(theme_to_evaluate)
                print(theme_to_evaluate, "->", thema_value)
                match_count += 1
                theme_evaluation_exact[theme_to_evaluate] = thema_code
    
    non_matches = list(set(all_data) - set(matches))
    print("all_data", len(all_data))
    print(all_data)
    print("matches", len(matches))
    print(matches)
    print("non_matches", len(non_matches))
    print(non_matches)

    return theme_evaluation_exact
theme_evaluation_naive = exact_match_evaluation(lowered_thema_dict)

sénégal -> sénégal
alaska -> alaska
bolivie -> bolivie
yémen -> yémen
iran -> iran
finlande -> finlande
guatemala -> guatemala
préhistoire -> préhistoire
oman -> oman
arts martiaux -> arts martiaux
tchécoslovaquie -> tchécoslovaquie
all_data 60
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatemala', 'préhistoire', 'astronomie', 'révolte', 'naissance/maternité', 'documentaire', 'oman', 'univers/système solaire', 'écrivain, écriture', 'histoire : 18e siècle', 'débarquement', 'aide humanitaire', 'rugby', 'histoire sans texte', 'poisson', "lecture de

In [63]:
import sys
from pprint import pprint
epsilon = sys.float_info.epsilon

def calculate_metrics(theme_evaluation):
 
    TP = epsilon # to avoid division by zero
    TP_array = []
    FP = 0
    FP_array = []
    for evaluation_theme_name, evaluation_theme_code in theme_evaluation.items():
        if evaluation_theme_name in themes_validation_set_ground_truth and evaluation_theme_code in themes_validation_set_ground_truth[evaluation_theme_name]:
            # print(key, value, themes_validation_set_ground_truth[key])
            TP_array.append((evaluation_theme_name, evaluation_theme_code, themes_validation_set_ground_truth[evaluation_theme_name]))
            TP += 1
        elif evaluation_theme_name in themes_validation_set_ground_truth and evaluation_theme_code not in themes_validation_set_ground_truth[evaluation_theme_name]:
            evaluation_theme_name_in_thema = thema_code_dict[evaluation_theme_code]["CodeDescription"] # name of the evaluated theme in thema
            validation_set_key = themes_validation_set_ground_truth[evaluation_theme_name][0]
            if validation_set_key != 'N/A' :
                validation_set_name = thema_code_dict[validation_set_key]["CodeDescription"] # name of the ground truth theme in thema
                FP_array.append(f"{evaluation_theme_name} -> {evaluation_theme_name_in_thema} vs φ={validation_set_name} | {evaluation_theme_code} vs φ={validation_set_key}")
            else:
                FP_array.append(f"{evaluation_theme_name} -> {evaluation_theme_name_in_thema} vs φ=N/A | {evaluation_theme_code} vs φ={validation_set_key}")


            FP += 1

            
    assert(FP == len(theme_evaluation) - 1) - TP
    FN = len(themes_validation_set_ground_truth) - TP  # no result counts as a non-match
    # a negative means we didn't classify something no TN we have to classify everything

    print("TP", TP)
    pprint(TP_array)
    print("FP", FP)
    pprint(FP_array)
    print("FN", FN)
    print("len(themes_validation_set_ground_truth)", len(themes_validation_set_ground_truth))

    accuracy = (TP / len(themes_validation_set_ground_truth))
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print("precision", precision)
    print("recall", recall)
    print("f1_score", f1_score)
    print("accuracy", accuracy)


calculate_metrics(theme_evaluation_naive)

NameError: name 'theme_evaluation_naive' is not defined

In [12]:
import Levenshtein
def levenstein_evaluation(thema_dict):
    theme_evaluation_levenstein = {}
    match_count = 0
    matches = []
    all_data = []

    for theme_to_evaluate in themes_validation_set_ground_truth.keys():
        # UPPER_DISTANCE_LIMIT = max(len(theme_to_evaluate) / 3, 0)
        UPPER_DISTANCE_LIMIT = 1
        all_data.append(theme_to_evaluate)
        min_distance = 100
        best_match = None
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"]
            distance = Levenshtein.distance(theme_to_evaluate, thema_value)
            if distance < min_distance and distance <= UPPER_DISTANCE_LIMIT:
                min_distance = distance
                best_match = thema_code
        if min_distance <= UPPER_DISTANCE_LIMIT and best_match is not None and theme_to_evaluate not in theme_evaluation_levenstein.values():
            matches.append(theme_to_evaluate)
            print(min_distance, theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"])
            match_count += 1
            theme_evaluation_levenstein[theme_to_evaluate] = best_match
    
    non_matches = list(set(all_data) - set(matches))
    print("all_data", len(all_data))
    print(all_data)
    print("matches", len(matches))
    print(matches)
    print("non_matches", len(non_matches))
    print(non_matches)

    return theme_evaluation_levenstein

theme_evaluation_levenstein = levenstein_evaluation(lowered_thema_dict)


0 sénégal -> sénégal
1 ours -> tours
0 alaska -> alaska
0 bolivie -> bolivie
0 yémen -> yémen
0 iran -> iran
0 finlande -> finlande
0 guatemala -> guatemala
0 préhistoire -> préhistoire
1 documentaire -> documentaires
0 oman -> oman
0 arts martiaux -> arts martiaux
0 tchécoslovaquie -> tchécoslovaquie
1 eau -> pau
all_data 60
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souffrance', 'nuit', 'guatemala', 'préhistoire', 'astronomie', 'révolte', 'naissance/maternité', 'documentaire', 'oman', 'univers/système solaire', 'écrivain, écriture', 'histoire : 18e siècle', 'débarq

In [ ]:
calculate_metrics(theme_evaluation_levenstein)

TP 11.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']), ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']), ('bolivie', '1KLSL', ['1KLSL', '1KLSL']), ('yémen', '1FBXY', ['1FBXY', '1FBXY']), ('iran', '1FBN', ['1FBN', '1FBN']), ('finlande', '1DNF', ['1DNF', '1DNF']), ('guatemala', '1KLCG', ['1KLCG', '1KLCG']), ('préhistoire', '3B', ['3B', '3B']), ('documentaire', 'ATFR', ['YN', 'ATFR']), ('oman', '1FBXM', ['1FBXM', '1FBXM']), ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK'])]
FP 3
[('1DDF-FR-GDA', 'tours', 'WNCF', 'vie sauvage : mammifères : généralités'), ('SRM', 'arts martiaux', 'YNWJ', 'documentaires jeunesse : arts martiaux'), ('1DDF-FR-BEA', 'pau', 'RBK', 'hydrologie et hydrosphère')]
FN 49.0
len(themes_validation_set_ground_truth) 60


(0.7857142857142857,
 0.18333333333333332,
 0.29729729729729726,
 0.18333333333333332)

## preprocessing: take 1st word, lower & remove special characters

In [14]:
preprocessed_thema_dict = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    preprocessed_thema_dict[code] = theme
    preprocessed_thema_dict[code]["CodeDescription"] = utils.preprocess_theme(theme["CodeDescription"])
    preprocessed_thema_dict[code]["CodeNotes"] = theme["CodeNotes"].lower()

preprocessed_thema_dict

{'A': {'CodeValue': 'A',
  'CodeDescription': 'arts',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4},
 'AB': {'CodeValue': 'AB',
  'CodeDescription': 'arts',
  'CodeNotes': '',
  'CodeParent': 'A',
  'IssueNumber': 1,
  'Modified': 1.5},
 'ABA': {'CodeValue': 'ABA',
  'CodeDescription': 'théorie',
  'CodeNotes': 'lié : qdtn',
  'CodeParent': 'AB',
  'IssueNumber': 1,
  'Modified': ''},
 'ABC': {'CodeValue': 'ABC',
  'CodeDescription': 'conservation',
  'CodeNotes': 'utilisez tous les codes a* pour les ouvrages sur la conservation, la préservation

In [15]:
evaluation_preprocessed = exact_match_evaluation(preprocessed_thema_dict)
calculate_metrics(evaluation_preprocessed)

sénégal -> sénégal
edition -> edition
alaska -> alaska
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
biographie -> biographie
bolivie -> bolivie
bolivie -> bolivie
yémen -> yémen
iran -> iran
semaine -> semaine
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
finlande -> finlande
nuit -> nuit
guatemala -> guatemala
guatemala -> guatemala
guatemala -> guatemala
guatemala -> guatemala
guatemala -> guatemala
guatemala -> guatemala
préhistoire -> préhistoire
astronomie -> astronomie
astronomie -> astronomie
astronomie -> astronomie
d

(0.5263157894736842,
 0.16666666666666666,
 0.25316455696202533,
 0.16666666666666666)

In [16]:
evaluation_preprocessed = levenstein_evaluation(preprocessed_thema_dict)
calculate_metrics(evaluation_preprocessed)

0 sénégal -> sénégal
0 edition -> edition
1 ours -> tours
0 alaska -> alaska
0 biographie -> biographie
0 bolivie -> bolivie
0 yémen -> yémen
1 dragon -> aragon
0 iran -> iran
0 semaine -> semaine
0 finlande -> finlande
1 peintre -> peinture
0 nuit -> nuit
0 guatemala -> guatemala
0 préhistoire -> préhistoire
0 astronomie -> astronomie
0 documentaire -> documentaire
0 oman -> oman
0 rugby -> rugby
1 poisson -> poissons
0 roller -> roller
0 tchécoslovaquie -> tchécoslovaquie
1 eau -> pau
0 londres -> londres
all_data 60
['balle, ballon', 'animaux domestiques', 'sénégal', 'livre-jeu', "atelier d'écriture", 'edition', 'ours', 'théâtre contemporain', 'maltraitance', 'transhumanisme', 'genre', 'alaska', 'biographie', 'bolivie', 'conte détourné, le loup et les 7 cabris', 'aventure et mystère', 'conte détourné', 'yémen', 'hippocampe', 'cow-boy', 'dragon', 'père noël', 'sida', 'souvenir/mémoire', 'pompier', 'iran', 'semaine', 'finlande', 'peintre', 'relation enfant, adulte', 'papillon', 'souff

(0.5833333333333334,
 0.23333333333333334,
 0.33333333333333337,
 0.23333333333333334)

# wiki2vec

In [20]:
from wikipedia2vec import Wikipedia2Vec
# download from https://wikipedia2vec.github.io/wikipedia2vec/pretrained/
# Load the pre-trained model
wiki2vec = Wikipedia2Vec.load('../book_themes_alignment_data/frwiki_20180420_300d.pkl')

# Find the most similar words to a given word
similar_words = wiki2vec.most_similar(wiki2vec.get_word('chien'), 10)

# Print the similar words
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

C:\Users\dre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[-0.3918146  -0.24315855 -0.29823107 ... -0.15211841  0.34864503
  -0.28443316]
 [-0.31431052 -0.05210092 -0.23165612 ...  0.33593094  0.21336684
  -0.2881443 ]
 [ 0.23390982 -0.00242238 -0.24155504 ...  0.10083728 -0.06711676
  -0.03356545]
 ...
 [-0.37162766 -0.5298218  -0.05777219 ... -0.2812191   0.15711616
   0.2854844 ]
 [-0.75576866 -0.14604965 -0.24931757 ... -0.32277733 -0.1752458
   0.3595142 ]
 [-0.4601417  -0.19192158 -0.281166   ... -0.36144966 -0.00496797
   0.08007599]] loaded from the file ../book_themes_alignment_data/frwiki_20180420_300d.pkl is not byte aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib

<Word chien>: 1.0000001192092896
<Word chiens>: 0.7172577381134033
<Word chiot>: 0.6617892980575562
<Word chat>: 0.658769428730011
<Word pudelpointer>: 0.6494312286376953
<Word chienne>: 0.6439096927642822
<Entity Chien de chasse>: 0.6328124403953552
<Word caniche>: 0.6294683814048767
<Word drahthaar>: 0.6275166869163513
<Word snuppy>: 0.6268122792243958


In [34]:
def align_lists_with_synonyms_wiki2vec(thema_dict, wiki2vec):

    theme_evaluation_wiki2vec = {}
    MINIMUM_COSINE_SIMILARITY = -1

    for theme_to_evaluate in themes_validation_set_ground_truth.keys():
        # Calculate the similarity between each pair of words
        max_similarity = MINIMUM_COSINE_SIMILARITY
        best_match = None
        for thema_code, thema_value in thema_dict.items():
            thema_value = thema_value["CodeDescription"] # name of thema theme
            try:
                word1 = wiki2vec.get_word_vector(theme_to_evaluate)
                word2 = wiki2vec.get_word_vector(thema_value)
                similarity = utils.cosine_similarity(word1, word2)
            except KeyError:
                similarity = MINIMUM_COSINE_SIMILARITY
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = thema_code
        if best_match is not None:
            print(theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"], max_similarity)
            theme_evaluation_wiki2vec[theme_to_evaluate] = best_match
        
    return theme_evaluation_wiki2vec

theme_evaluation_wiki2vec = align_lists_with_synonyms_wiki2vec(preprocessed_thema_dict, wiki2vec)

sénégal -> sénégal 0.99999994
edition -> edition 1.0
ours -> tigre 0.47072738
maltraitance -> négligence 0.56435037
transhumanisme -> survivalisme 0.5340962
genre -> espèces 0.50099796
alaska -> alaska 1.0000001
biographie -> biographie 1.0000001
bolivie -> bolivie 1.0
yémen -> yémen 1.0
hippocampe -> chats 0.3433567
dragon -> tigre 0.5321037
sida -> virus 0.56429756
pompier -> infirmier 0.5436603
iran -> iran 1.0000001
semaine -> semaine 1.0000001
finlande -> finlande 1.0000001
peintre -> peinture 0.59788054
papillon -> natation 0.3748513
souffrance -> douleur 0.73902035
nuit -> nuit 1.0000001
guatemala -> guatemala 0.99999994
préhistoire -> préhistoire 1.0000001
astronomie -> astronomie 1.0
révolte -> mutinerie 0.65357226
documentaire -> documentaire 1.0
oman -> oman 1.0000001
débarquement -> invasion 0.5011468
rugby -> rugby 1.0
poisson -> poissons 0.7376943
roller -> roller 1.0
licorne -> magie 0.41852835
tchécoslovaquie -> tchécoslovaquie 1.0
eau -> hydraulique 0.4844982
londres -

In [50]:
calculate_metrics(theme_evaluation_wiki2vec)

TP 14.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('edition', 'KNTP1', ['KNTP1', 'KNTP1']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('yémen', '1FBXY', ['1FBXY', '1FBXY']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('astronomie', 'PG', ['PG', 'YPMP51']),
 ('documentaire', 'YN', ['YN', 'ATFR']),
 ('oman', '1FBXM', ['1FBXM', '1FBXM']),
 ('tchécoslovaquie', '1QBDK', ['1QBDK', '1QBDK']),
 ('londres', '1DDU-GB-ESL', ['JPT', '1DDU-GB-ESLF', '1DDU-GB-ESL'])]
FP 21
['ours -> tigre vs φ=vie , 1FBZB vs φ=WNCF',
 'maltraitance -> négligence vs φ=jeunesse , LNVC vs φ=YXQD',
 'transhumanisme -> survivalisme vs φ=technologies , VSY vs φ=UBJ',
 'genre -> espèces vs φ=théorie , RNKH1 vs φ=JBSF',
 'biographie -> biographie vs φ=biographie , D vs φ=DNB',
 'hippocampe -> chats vs φ=vie , WNGC vs φ=WNCS1',
 'dragon -> ti

# doc2vec

In [2]:
import spacy
# from spacy.lang.fr.examples import sentences
spacy_nlp = spacy.load('fr_core_news_lg')

## testing of similarities

In [3]:
doc = spacy_nlp("La grande querelle que nous avons cherchée à éviter est arrivée.")
doc_vector = doc.vector
doc_vector
for token in doc:
    print(token.text, token.pos_, token.dep_)
# keep noun, verb, adjective

La DET det
grande ADJ amod
querelle NOUN nsubj
que PRON obj
nous PRON nsubj
avons AUX aux:tense
cherchée VERB acl:relcl
à ADP mark
éviter VERB xcomp
est AUX cop
arrivée VERB ROOT
. PUNCT punct


In [13]:
import pandas as pd
# Define the themes
theme_uno = "amour, relations, futur"
theme_dos = "relations amoureuses"
theme_tres = "relations parentales"
theme_quatro = "relations amicales"

# Process the themes with spaCy
doc_uno = spacy_nlp(theme_uno)
doc_dos = spacy_nlp(theme_dos)
doc_tres = spacy_nlp(theme_tres)
doc_quatro = spacy_nlp(theme_quatro)

# Calculate the similarity scores
similarity_uno_dos = doc_uno.similarity(doc_dos)
similarity_uno_tres = doc_uno.similarity(doc_tres)
similarity_uno_cuatro = doc_uno.similarity(doc_quatro)
similarity_dos_tres = doc_dos.similarity(doc_tres)
similarity_dos_cuatro = doc_dos.similarity(doc_quatro)
similarity_tres_cuatro = doc_tres.similarity(doc_quatro)

# Print the similarity scores
print("Similarity between theme_uno and theme_dos:", similarity_uno_dos)
print("Similarity between theme_uno and theme_tres:", similarity_uno_tres)
print("Similarity between theme_uno and theme_quatro:", similarity_uno_cuatro)
print("Similarity between theme_dos and theme_tres:", similarity_dos_tres)
print("Similarity between theme_dos and theme_quatro:", similarity_dos_cuatro)
print("Similarity between theme_tres and theme_quatro:", similarity_tres_cuatro)

# Create a dictionary with the similarity scores
similarity_scores = {
    'theme_uno': {
        'theme_dos': similarity_uno_dos,
        'theme_tres': similarity_uno_tres,
        'theme_quatro': similarity_uno_cuatro
    },
    'theme_dos': {
        'theme_uno': similarity_uno_dos,
        'theme_tres': similarity_dos_tres,
        'theme_quatro': similarity_dos_cuatro
    },
    'theme_tres': {
        'theme_uno': similarity_uno_tres,
        'theme_dos': similarity_dos_tres,
        'theme_quatro': similarity_tres_cuatro
    },
    'theme_quatro': {
        'theme_uno': similarity_uno_cuatro,
        'theme_dos': similarity_dos_cuatro,
        'theme_tres': similarity_tres_cuatro
    }
}

# Create a DataFrame from the similarity scores dictionary
df_similarity = pd.DataFrame(similarity_scores)

# Display the DataFrame
print(df_similarity)


Similarity between theme_uno and theme_dos: 0.7164997982353112
Similarity between theme_uno and theme_tres: 0.6306297939840912
Similarity between theme_uno and quatro: 0.6064372921366171
Similarity between theme_dos and theme_tres: 0.9209783377591776
Similarity between theme_dos and quatro: 0.9384199697268918
Similarity between theme_tres and quatro: 0.9149825085628936
            theme_uno  theme_dos  theme_tres    quatro
theme_dos    0.716500        NaN    0.920978  0.938420
theme_tres   0.630630   0.920978         NaN  0.914983
quatro       0.606437   0.938420    0.914983       NaN
theme_uno         NaN   0.716500    0.630630  0.606437


## subset of thema dict for testing purpose

In [45]:
existing_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}

# Create a new dictionary with elements from 'a' to 'c'
subset_dict = {key: value for key, value in existing_dict.items() if key in ['a', 'b', 'c']}

print(subset_dict)


{'A': {'CodeValue': 'A', 'CodeDescription': 'Arts', 'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*', 'CodeParent': '', 'IssueNumber': 1, 'Modified': 1.4}, 'AB': {'CodeValue': 'AB', 'CodeDescription': 'Arts : généralités', 'CodeNotes': '', 'CodeParent': 'A', 'IssueNumber': 1, 'Modified': 1.5}, 'ABC': {'CodeValue': 'ABC', 'CodeDescription': 'Conservation, restauration et entretien d’œuvres d’art', 'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages sur la conservation, la préservation, la rénovation, la restauration ou l’entretien de tout type d’art, y compris les bâtiments, les structures, les sculptur

In [59]:
thema_dict_cached_nlp = {}
for code, theme in thema_code_dict.items():
    # Preprocess the code description
    thema_dict_cached_nlp[code] = theme
    thema_dict_cached_nlp[code]["NLP_CodeDescription"] = spacy_nlp(theme["CodeDescription"].lower())
    thema_dict_cached_nlp[code]["NLP_CodeNotes"] = spacy_nlp(theme["CodeNotes"].lower())

thema_dict_cached_nlp

{'A': {'CodeValue': 'A',
  'CodeDescription': 'Arts',
  'CodeNotes': 'Utilisez tous les codes A* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. Préférez les codes WF* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) A*. Utilisez au besoin tous les codes A* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* et historiques 3*',
  'CodeParent': '',
  'IssueNumber': 1,
  'Modified': 1.4,
  'NLP_CodeDescription': arts,
  'NLP_CodeNotes': utilisez tous les codes a* pour les ouvrages spécialisés et généraux, qu’ils soient richement illustrés ou majoritairement textuel. préférez les codes wf* pour les ouvrages liés à un loisir ou un passe-temps, en le complétant au besoin par le(s) code(s) a*. utilisez au besoin tous les codes a* avec d’autres codes et qualificateurs, particulièrement avec les qualificateurs de style 6*, de lieux 1* e

In [57]:
thema_dict_cached_nlp['A']['NLP_CodeDescription'].vector

array([-2.4444  ,  3.6131  , -1.5427  ,  0.19821 , -4.9109  , -0.92411 ,
       -3.4663  , -4.7047  ,  2.1716  ,  2.2285  , -2.9157  ,  1.3476  ,
       -0.8261  , -1.6747  , -0.78852 ,  1.4686  , -1.0619  ,  2.5078  ,
        2.4267  , -1.4342  ,  2.0273  , -1.1112  ,  0.93525 ,  1.0275  ,
        1.7172  ,  4.2398  ,  6.6417  , -2.7434  , -4.9158  , -3.1832  ,
       -3.0791  ,  0.18367 , -3.1694  ,  0.39731 ,  2.5735  ,  1.5541  ,
       -0.062134, -1.1823  ,  1.6127  , -0.53979 ,  3.74    , -2.8257  ,
       -0.93448 , -0.22003 ,  2.6798  , -0.62695 , -3.1505  ,  5.4204  ,
        4.7317  , -1.8757  ,  1.4825  ,  1.0483  ,  2.079   , -0.69718 ,
        3.3999  ,  4.9525  ,  2.9798  ,  2.7246  ,  4.7729  ,  3.9539  ,
       -2.1769  , -1.402   , -0.4705  , -0.75873 , -1.362   ,  0.42603 ,
        2.964   ,  3.6214  , -2.7319  , -4.4059  , -2.6467  ,  2.4078  ,
        2.1962  , -1.1723  , -0.75522 ,  3.6917  ,  2.5882  ,  1.1995  ,
        1.262   ,  0.53243 ,  2.8513  ,  2.2561  , 

## align with CodeDescription = theme name

In [58]:
def align_lists_with_synonyms_spacy(thema_dict, spacy_nlp):

    theme_evaluation_model = {}
    MINIMUM_COSINE_SIMILARITY = -1

    for theme_to_evaluate in themes_validation_set_ground_truth.keys():
        # Calculate the similarity between each pair of words
        max_similarity = MINIMUM_COSINE_SIMILARITY
        best_match = None
        theme_to_evaluate_nlp = spacy_nlp(theme_to_evaluate)
        for thema_code, thema_value in thema_dict.items():
            thema_value_nlp = thema_value["NLP_CodeDescription"] # name of thema theme  
            similarity = theme_to_evaluate_nlp.similarity(thema_value_nlp)
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = thema_code
        if best_match is not None:
            print(theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"], max_similarity)
            theme_evaluation_model[theme_to_evaluate] = best_match
        
    return theme_evaluation_model


theme_evaluation_spacy_code_description = align_lists_with_synonyms_spacy(thema_code_dict, spacy_nlp) 

C:\Users\dre\AppData\Local\Temp\ipykernel_41520\2624200095.py:17: UserWarning:

[W008] Evaluating Doc.similarity based on empty vectors.



balle, ballon -> Activités de plein air : pêche, chasse, tir 0.5267925784036915
animaux domestiques -> Pratiques médicales supplémentaires pour les animaux 0.7596218669989263
sénégal -> Sénégal 0.7979945091044219
livre-jeu -> Techno-thriller 0.6048049815746949
atelier d'écriture -> Techniques d’écriture de scénarios 0.7652673587783552
edition -> Illustration 0.6772901198722592
ours -> Documentaires jeunesse : ours 0.49510530838693956
théâtre contemporain -> Théâtre 0.7159894188257122
maltraitance -> Violence domestique 0.7211113417514288
transhumanisme -> Jaïnisme 0.7670071862653235
genre -> Relatif aux rôles de genre non stéréotypés 0.4531132376816016
alaska -> Alaska 0.6805375878048374
biographie -> Historiographie 0.7547585980179244
bolivie -> Bolivie 0.8095498913660971
conte détourné, le loup et les 7 cabris -> Southampton et le Solent 0.7310696234888956
aventure et mystère -> Bonne aventure et divination 0.8516056869710872
conte détourné -> Réalisme magique 0.6118471826403723
yéme

In [66]:
calculate_metrics(theme_evaluation_spacy_code_description)

TP 10.0
[('sénégal', '1HFDS', ['1HFDS', '1HFDS']),
 ('alaska', '1KBB-US-WPNA', ['1KBB-US-WPNA', '1KBB-US-WPNA']),
 ('bolivie', '1KLSL', ['1KLSL', '1KLSL']),
 ('père noël', '5HPD', ['5HPD', 'N/A']),
 ('sida', 'MJCJ2', ['MJCJ2', 'MJCJ2']),
 ('iran', '1FBN', ['1FBN', '1FBN']),
 ('finlande', '1DNF', ['1DNF', '1DNF']),
 ('guatemala', '1KLCG', ['1KLCG', '1KLCG']),
 ('préhistoire', '3B', ['3B', '3B']),
 ('aide humanitaire', 'JKSR', ['JKSR', 'JKSR'])]
FP 50
['balle, ballon -> Activités de plein air : pêche, chasse, tir vs φ=Sports / '
 'jeux de ballon | SV vs φ=SF',
 'animaux domestiques -> Pratiques médicales supplémentaires pour les animaux '
 'vs φ=Animaux domestiques et de compagnie | MZX vs φ=WNG',
 'livre-jeu -> Techno-thriller vs φ=Livres d’histoires illustrés : imagination '
 'et jeux | FHK vs φ=YBCS2',
 "atelier d'écriture -> Techniques d’écriture de scénarios vs φ=Arts de "
 'l’écriture et techniques de rédaction | CBVS vs φ=CBW',
 'edition -> Illustration vs φ=Edition et commerce du

## align with codeNotes = long text that describes theme

In [67]:
def align_lists_with_synonyms_spacy(thema_dict, spacy_nlp):

    theme_evaluation_model = {}
    MINIMUM_COSINE_SIMILARITY = -1

    for theme_to_evaluate in themes_validation_set_ground_truth.keys():
        # Calculate the similarity between each pair of words
        max_similarity = MINIMUM_COSINE_SIMILARITY
        best_match = None
        theme_to_evaluate_nlp = spacy_nlp(theme_to_evaluate)
        for thema_code, thema_value in thema_dict.items():
            thema_value_nlp = thema_value["NLP_CodeNotes"] # name of thema theme  
            similarity = theme_to_evaluate_nlp.similarity(thema_value_nlp)
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = thema_code
        if best_match is not None:
            print(theme_to_evaluate, "->", thema_dict[best_match]["CodeDescription"], max_similarity)
            theme_evaluation_model[theme_to_evaluate] = best_match
        
    return theme_evaluation_model


theme_evaluation_spacy_code_notes = align_lists_with_synonyms_spacy(thema_code_dict, spacy_nlp) 

C:\Users\dre\AppData\Local\Temp\ipykernel_41520\1874898275.py:13: UserWarning:

[W008] Evaluating Doc.similarity based on empty vectors.



balle, ballon -> Sports aériens 0.5143243342540329
animaux domestiques -> Cuisine végétarienne et le végétarisme 0.6747852536938753
sénégal -> Nantes 0.3473832954979771
livre-jeu -> Kirkenes 0.5236881525242607
atelier d'écriture -> Auvergne-Rhône-Alpes: lieux d’intérêt 0.6940483547634733
edition -> Civilisations de la vallée de l’Indus 0.44469665866592584
ours -> Antiquités et objets anciens et de collection : jouets, jeux, poupées et modèles 0.4481108876676137
théâtre contemporain -> Installation artistique 0.5743395586952929
maltraitance -> Droit pénal : infractions contre les personnes 0.5674047023559341
transhumanisme -> Philosophie et théorie de l’éducation 0.6865510596390816
genre -> Fiction jeunesse : générale, moderne et contemporaine 0.5452990021595834
alaska -> Newcastle et Gateshead 0.23560908305511286
biographie -> Economie populaire 0.34160084045637723
bolivie -> Pérou: région andine 0.23722524024829741
conte détourné, le loup et les 7 cabris -> Gymnastique 0.8163241977896

In [68]:
calculate_metrics(theme_evaluation_spacy_code_notes)

TP 2.0
[('nuit', 'YBLJ', ['YBLJ', 'YBLJ']),
 ('aide humanitaire', 'JKSR', ['JKSR', 'JKSR'])]
FP 58
['balle, ballon -> Sports aériens vs φ=Sports / jeux de ballon | SMC vs φ=SF',
 'animaux domestiques -> Cuisine végétarienne et le végétarisme vs φ=Animaux '
 'domestiques et de compagnie | WBJ vs φ=WNG',
 'sénégal -> Nantes vs φ=Sénégal | 1DDF-FR-RAB vs φ=1HFDS',
 'livre-jeu -> Kirkenes vs φ=Livres d’histoires illustrés : imagination et '
 'jeux | 1DNN-NO-DFB vs φ=YBCS2',
 "atelier d'écriture -> Auvergne-Rhône-Alpes: lieux d’intérêt vs φ=Arts de "
 'l’écriture et techniques de rédaction | 1DDF-FR-XAZ vs φ=CBW',
 'edition -> Civilisations de la vallée de l’Indus vs φ=Edition et commerce du '
 'livre | 1QBFB vs φ=KNTP1',
 'ours -> Antiquités et objets anciens et de collection : jouets, jeux, '
 'poupées et modèles vs φ=Vie sauvage : mammifères : généralités | WCW vs '
 'φ=WNCF',
 'théâtre contemporain -> Installation artistique vs φ=Théâtre moderne et '
 'contemporain (à partir de 1900) | 